# **<span style="font-family: 'Palatino Linotype', serif;">🐢🐇 Comparando as performances</span>**
----
*<span style="font-family: 'Angilla Tattoo'"> "Testaram minha performance em 12 realidades. Em todas, fui banida por ser apelona."⚡🎮🚷 </span>*

*<span style="font-family: 'Angilla Tattoo'"> - Maga Patolina </span>*

<div align="center">
    <img src = "Maga dimensoes.png" alt = "Maga nas realidades" width = 300>
</div>

----
 **Objetivo:** Nesse notebook, irei comparar o tempo de resolução de três algoritmos de otimização (busca aletória, busca em grade e algoritmo genético) no problema das caixas binárias. Para isso, irei selecionar 10 valores distintos de caixas e rodar a mesma busca repetidas vezes para encontrar as médias de  tempo. 
 
---

In [2]:
from funcoes_2 import funcao_objetivo_pop_cb as funcao_objetivo
from funcoes_2 import populacao_cb as cria_populacao
from funcoes_2 import selecao_roleta_max as funcao_selecao
from funcoes_2 import cruzamento_ponto_simples as funcao_cruzamento
from funcoes_2 import mutacao_simples_cb as funcao_mutacao
from funcoes_1 import funcao_objetivo_cb as funcao_objetivo_ba
from funcoes_1 import gene_cb as gene
from funcoes_1 import cria_candidato_cb as cria_candidato 
from pprint import pprint
import random
import time
import itertools
import statistics

----

### 🕵️‍♀️ **Busca aleatória**

In [3]:
lista_valores_candidatos = list(range(2,12))
print(lista_valores_candidatos)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [4]:
sorteios = 50

In [10]:
NUM_REPETICOES = range(20)
tempos = []

for num in NUM_REPETICOES:
    tempo_inicial = time.time()

    for tamanho in lista_valores_candidatos:
        for i in range(sorteios):
            candidato = cria_candidato(tamanho)  
            fun_obj = funcao_objetivo_ba(candidato)
            # print(f"Tamanho {tamanho}, Sorteio {i}: {fun_obj}")

    tempo_final = time.time()

    tempo_execucao = tempo_final - tempo_inicial
    tempos.append(tempo_execucao)

media_tempos = statistics.mean(tempos)

print("O tempo médio parao algoritmo de busca aletória foi", media_tempos)

O tempo médio parao algoritmo de busca aletória foi 0.0031563758850097655


Para os menores de valores de n dos candidatos, foi possível encontrar a resposta correta do problema. Quanto mais os valores aumentam, mais difícil é para o problema convergir e mais distantes do maior resultado possível se tornam os resultados obtidos.

----

### 🔲 **Busca em grade**

In [11]:
NUM_REPETICOES = range(20)
tempos = []

for num in NUM_REPETICOES:
    tempo_inicial = time.time()

    for tamanho in lista_valores_candidatos:
        todas_combinacoes = list(itertools.product([0, 1], repeat=tamanho))

    for candidato in todas_combinacoes:
        candidato = list(candidato)
        func_obj_bg = funcao_objetivo_ba(candidato)
        # print(f"Tamanho {tamanho}, {func_obj_bg}")

    tempo_final = time.time()
    tempo_execucao = tempo_final - tempo_inicial
    tempos.append(tempo_execucao)

media_tempos = statistics.mean(tempos)

print("O tempo médio parao algoritmo de busca aletória foi", media_tempos)

O tempo médio parao algoritmo de busca aletória foi 0.00141448974609375


Nesse caso, como testamos todas as combinações possíveis, garantimos que iremos encontrar o melhor resultado possível para qualquer número de n. 

----

### 🧬 **Algoritmo genético**

In [7]:
TAMANHO_POPULACAO = 10
NUM_GERACOES = 50
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.05

In [8]:
for tamanho in lista_valores_candidatos:
    populacao = cria_populacao(TAMANHO_POPULACAO, tamanho)
    print(f"\nTamanho: {tamanho}")
    pprint(populacao)


Tamanho: 2
[[0, 1], [1, 1], [1, 0], [0, 1], [0, 0], [1, 1], [0, 1], [0, 1], [1, 0], [1, 1]]

Tamanho: 3
[[1, 0, 1],
 [0, 1, 1],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 1],
 [1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [1, 0, 1],
 [1, 1, 0]]

Tamanho: 4
[[1, 1, 0, 1],
 [0, 1, 1, 0],
 [1, 1, 0, 1],
 [0, 0, 1, 1],
 [1, 1, 1, 0],
 [1, 0, 1, 1],
 [1, 0, 1, 1],
 [1, 1, 1, 0],
 [0, 1, 0, 1],
 [0, 1, 1, 0]]

Tamanho: 5
[[1, 1, 1, 1, 1],
 [1, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [1, 0, 0, 1, 0],
 [1, 0, 1, 0, 0],
 [0, 0, 0, 1, 1],
 [1, 0, 1, 0, 1]]

Tamanho: 6
[[0, 0, 0, 1, 0, 1],
 [1, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 1],
 [1, 0, 1, 0, 1, 1],
 [1, 1, 1, 1, 0, 0],
 [1, 1, 1, 0, 1, 0],
 [1, 1, 1, 1, 1, 1],
 [1, 0, 0, 1, 0, 1],
 [0, 0, 1, 0, 0, 1],
 [0, 0, 0, 1, 0, 0]]

Tamanho: 7
[[0, 1, 1, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 1, 1, 1, 1],
 [0, 0, 1, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 1, 1, 0, 0, 0, 0],
 [1, 1, 0, 0, 1, 1, 0],
 [1, 1, 0, 1, 1, 0,

In [12]:
todas_populacoes = {}
lista_valores_candidatos = list(range(2, 12)) 
NUM_REPETICOES = range(20)
tempos = []

for tamanho in lista_valores_candidatos:
    todas_populacoes[tamanho] = cria_populacao(TAMANHO_POPULACAO, tamanho)

tempo_inicial = time.time() 

NUM_REPETICOES = range(20)
tempos = []

for num in NUM_REPETICOES:

    for tamanho, populacao_inicial in todas_populacoes.items():
        # print(f"\nRodando algoritmo genético para população com {tamanho} genes")
        populacao = populacao_inicial.copy()
        hall_da_fama = []

        for n in range(NUM_GERACOES):
            
            # Seleção
            fitness = funcao_objetivo(populacao)        
            selecionados = funcao_selecao(populacao, fitness)
            
            # Cruzamento
            proxima_geracao = []
            for pai, mae in zip(selecionados[::2], selecionados[1::2]):
                individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
                proxima_geracao.append(individuo1)
                proxima_geracao.append(individuo2)
            
            # Mutação
            funcao_mutacao(proxima_geracao, CHANCE_DE_MUTACAO)
            
            # Atualização do hall da fama
            fitness = funcao_objetivo(proxima_geracao)
            maior_fitness = max(fitness)
            indice = fitness.index(maior_fitness)
            hall_da_fama.append(proxima_geracao[indice])    
            
            # Encerramento
            populacao = proxima_geracao

        # print(f"Melhor indivíduo para tamanho {tamanho}: {hall_da_fama[-1]}")

    tempo_final = time.time()
    tempo_execucao = tempo_final - tempo_inicial

    tempos.append(tempo_execucao)

media_tempos = statistics.mean(tempos)

print("O tempo médio parao algoritmo de busca aletória foi", media_tempos)

O tempo médio parao algoritmo de busca aletória foi 0.0951123833656311


----

### 💡 **Analisando os resultados** 

Podemos observar que o algoritmo de busca aleatória é o que funciona em menor tempo, porém, dificilmente converge para valores maiores de n. O algoritmo em busca grade garante que o melhor resulatado possível será encontrado, porém leva um tempo maior para fazer isso, que aumenta exponencialmente com a quantidade de dados, e é mais custoso computacionalmente. Já os algoritmos genéticos, nesse caso, foi o mais longo, porém, o tempo para encontrar a resposta cresce mais lentamente conforme os dados aumentam, e em geral é posspivel convergir para valores maiores de n. Assim, podemos concluir que os algortimos genéticos são uma boa estratégia de otimização. 

----
### 📚 Referências:

OPENAI. ChatGPT (versão GPT-4) [programa de computador]. Disponível em: <https://chatgpt.com/share/6807a849-a330-8005-a38e-db909257b2e9>. Acesso em: 18 abr. 2025.